<a href="https://colab.research.google.com/github/dimidagd/gists/blob/main/Subclassing_Models_forward_pass%2C_load_weights_and_register_hooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


In [8]:
RESNET_PTH_FILE = "/root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth"

In [122]:
!cat /root/.cache/torch/hub/pytorch_vision_v0.9.0/torchvision/models/resnet.py | grep -A 5 "class ResNet"
!echo -e ".\n.\n.\n.\n.\n.\n.\n"
!cat /root/.cache/torch/hub/pytorch_vision_v0.9.0/torchvision/models/resnet.py | grep -A 50 "class ResNet" | grep -A 2 -B 2 "self.conv1"
!echo -e ".\n.\n.\n.\n.\n.\n.\n"

class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
.
.
.
.
.
.
.

        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
.
.
.
.
.
.
.



In [126]:
class Hook:
    def __init__(self):
        self._input = None
        self._output = None

    def __call__(self, module, input, output):
        print(len(input))
        self._input = input[0].detach()
        self._output = output[0].detach()

    def reset(self):
        self._input, self._output = None, None

    @property
    def input(self):
        return self._input

    @property
    def output(self):
        return self._output

# Define a hook function to capture the output
class OutputHook:
    def __init__(self,module):
        self.module = module
        self._state = Hook()
        self.hook = NotImplemented
    def __enter__(self):
        self.hook = self.module.register_forward_hook(self._state)
        return self
    def __exit__(self, exc_type, exc_value, traceback):
        print(f"Releasing")
        self._state.reset()
        self.hook.remove()
        # Perform cleanup or resource release

In [127]:
models.ResNet(block=models.resnet.BasicBlock, layers=[2, 2, 2, 2], num_classes=1000).conv1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [128]:
import torch
import torch.nn as nn
import torchvision.models as models

# Subclass the pretrained model directly
class MyModel(models.ResNet):
    def __init__(self):
        # Call the constructor of the parent class (ResNet)
        super(MyModel, self).__init__(block=models.resnet.BasicBlock, layers=[2, 2, 2, 2], num_classes=1000)
        # You can customize the initialization here if needed

    def custom_forward(self, x):
        # Add your custom method logic here
        module_to_be_hooked = getattr(self, "conv1")
        with OutputHook(module_to_be_hooked) as hook: # self.conv1 created by models.ResNet __init__
          final_output = self.forward(x)
          module_input = hook._state.input
        return final_output, module_input

# Create an instance of your subclassed model
my_model = MyModel()

# Download the pretrained weights
pretrained_weights = torch.load(RESNET_PTH_FILE)

# Load the pretrained weights into your model
my_model.load_state_dict(pretrained_weights, strict=False)

# Now your model is initialized with the pretrained weights

<All keys matched successfully>

In [130]:
model_output, hooked_module_output = my_model.custom_forward(torch.rand(13,3,100,100))

print("model output",model_output.shape)
print("hooked module output",hooked_module_output.shape)

1
Releasing
model output torch.Size([13, 1000])
hooked module output torch.Size([13, 3, 100, 100])
